## Import Packages and Data Set

In [4]:
import chess.pgn
import numpy as np
import pandas as pd
import pickle

In [5]:
games = open("lichess_db_standard_rated_2014-09.pgn", encoding="utf8", errors='ignore')

<br>

## Data Cleaning and Filtering

In [6]:
# Read in each game and format
result = {}
i = 0
while True:
    i += 1
    game = chess.pgn.read_game(games)
    if game is None:
        break

    headers = dict(game.headers)
    headers["moves"] = game.board().variation_san(game.mainline_moves())
    headers["move_num"] = headers["moves"].count('.')

    result["Game{}".format(i)] = headers

In [7]:
# Create data frame of results
results = pd.DataFrame(result)
df = results.transpose()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000056 entries, Game1 to Game1000056
Data columns (total 21 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   Event            1000056 non-null  object
 1   Site             1000056 non-null  object
 2   Date             1000056 non-null  object
 3   Round            1000056 non-null  object
 4   White            1000056 non-null  object
 5   Black            1000056 non-null  object
 6   Result           1000056 non-null  object
 7   BlackElo         1000056 non-null  object
 8   BlackRatingDiff  999325 non-null   object
 9   ECO              1000056 non-null  object
 10  Opening          1000056 non-null  object
 11  Termination      1000056 non-null  object
 12  TimeControl      1000056 non-null  object
 13  UTCDate          1000056 non-null  object
 14  UTCTime          1000056 non-null  object
 15  WhiteElo         1000056 non-null  object
 16  WhiteRatingDiff  999325 non-null 

In [8]:
# Create variable for winner
df['winner'] = np.where(df['Result'] == '1-0', 'white', np.where(df['Result'] == '0-1', 'black', 'draw'))

In [9]:
# Remove games that were a draw
chessdf = df[df['winner']!='draw']
chessdf.head()

Event                          Site        Date Round  \
Game1  Rated Bullet game  https://lichess.org/1Vimq9SL  ????.??.??     ?   
Game2   Rated Blitz game  https://lichess.org/PK5H93NR  ????.??.??     ?   
Game3  Rated Bullet game  https://lichess.org/br0KfpXd  ????.??.??     ?   
Game4   Rated Blitz game  https://lichess.org/4CnlXUB0  ????.??.??     ?   
Game5  Rated Bullet game  https://lichess.org/X9Q96D1l  ????.??.??     ?   

             White       Black Result BlackElo BlackRatingDiff  ECO  ...  \
Game1       KACHAL  justplaybi    0-1     1823             +12  A06  ...   
Game2     mustroll       pelao    1-0     1658             -13  A00  ...   
Game3  luciano2000     amnezia    1-0     1790             -12  B01  ...   
Game4      martinz       HighP    1-0     1650             -13  C42  ...   
Game5      Thoth33  JaiSkiesNY    1-0     1545              -7  D31  ...   

      TimeControl     UTCDate   UTCTime WhiteElo WhiteRatingDiff  \
Game1       120+0  2014.08.31  22:00:33     1833             -11   
Game2       240+2  2014.08.31  22:00:53     1600             +13   
Game3        60+0  2014.08.31  22:00:03     1754             +13   
Game4       300+0  2014.08.31  22:00:17     1601             +13   
Game5        60+0  2014.08.31  22:00:04     1657             +10   

                                                   moves move_num BlackTitle  \
Game1  1. e4 d5 2. Nf3 dxe4 3. Ne5 Nf6 4. d4 exd3 5. ...       32        NaN   
Game2  1. b4 e5 2. Bb2 d6 3. c3 Bf5 4. d3 Nf6 5. e4 B...       67        NaN   
Game3  1. e4 d5 2. exd5 Qxd5 3. Nc3 Qa5 4. Nf3 Nf6 5....       39        NaN   
Game4  1. e4 e5 2. Nf3 Nf6 3. Nc3 d6 4. Bc4 Be6 5. Qe...       26        NaN   
Game5  1. d4 e6 2. c4 c6 3. Nc3 d5 4. cxd5 cxd5 5. e4...       29        NaN   

      WhiteTitle winner  
Game1        NaN  black  
Game2        NaN  white  
Game3        NaN  white  
Game4        NaN  white  
Game5        NaN  white  

[5 rows x 22 columns]

In [30]:
# Save copy of Sep 2014 data
with open('2014_09_chess_data.pickle', 'wb') as to_write:
        pickle.dump(chessdf, to_write)

In [10]:
# Remove games with unknown Elo ratings
has_white_elo = chessdf[chessdf['WhiteElo'] != '?']
has_black_elo = has_white_elo[has_white_elo['BlackElo'] != '?']

In [11]:
# Restrict data to just necessary columns
chess_games = has_black_elo[['BlackElo', 'WhiteElo', 'moves', 'move_num', 'winner']]
chess_games.head()

BlackElo WhiteElo                                              moves  \
Game1     1823     1833  1. e4 d5 2. Nf3 dxe4 3. Ne5 Nf6 4. d4 exd3 5. ...   
Game2     1658     1600  1. b4 e5 2. Bb2 d6 3. c3 Bf5 4. d3 Nf6 5. e4 B...   
Game3     1790     1754  1. e4 d5 2. exd5 Qxd5 3. Nc3 Qa5 4. Nf3 Nf6 5....   
Game4     1650     1601  1. e4 e5 2. Nf3 Nf6 3. Nc3 d6 4. Bc4 Be6 5. Qe...   
Game5     1545     1657  1. d4 e6 2. c4 c6 3. Nc3 d5 4. cxd5 cxd5 5. e4...   

      move_num winner  
Game1       32  black  
Game2       67  white  
Game3       39  white  
Game4       26  white  
Game5       29  white

In [12]:
# Create int variables for Elo ratings 
chess_games['Black_Elo_Num'] = chess_games['BlackElo'].astype(int)
chess_games['White_Elo_Num'] = chess_games['WhiteElo'].astype(int)
chess_games.info()

<ipython-input-12-06d89158ddd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_games['Black_Elo_Num'] = chess_games['BlackElo'].astype(int)


<class 'pandas.core.frame.DataFrame'>
Index: 962968 entries, Game1 to Game1000056
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   BlackElo       962968 non-null  object
 1   WhiteElo       962968 non-null  object
 2   moves          962968 non-null  object
 3   move_num       962968 non-null  object
 4   winner         962968 non-null  object
 5   Black_Elo_Num  962968 non-null  int64 
 6   White_Elo_Num  962968 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 58.8+ MB


<ipython-input-12-06d89158ddd4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_games['White_Elo_Num'] = chess_games['WhiteElo'].astype(int)


In [13]:
# Look at summary of Elo ratings
chess_games.describe()

Black_Elo_Num  White_Elo_Num
count  962968.000000  962968.000000
mean     1626.560238    1631.789059
std       239.517448     236.951933
min       737.000000     751.000000
25%      1468.000000    1473.000000
50%      1623.000000    1630.000000
75%      1784.000000    1789.000000
max      2711.000000    2708.000000

In [14]:
# Restrict data set to only Elo ratings over 1800
over_1800 = chess_games[(chess_games['White_Elo_Num'] >= 1800) & (chess_games['Black_Elo_Num'] >= 1800)]
over_1800.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137126 entries, Game1 to Game1000055
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   BlackElo       137126 non-null  object
 1   WhiteElo       137126 non-null  object
 2   moves          137126 non-null  object
 3   move_num       137126 non-null  object
 4   winner         137126 non-null  object
 5   Black_Elo_Num  137126 non-null  int64 
 6   White_Elo_Num  137126 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 8.4+ MB


In [15]:
# Restrict data so at least one player has an Elo rating of 2100 or above
both_1800_one_2100 = over_1800[(over_1800['White_Elo_Num'] >= 2100) | (over_1800['Black_Elo_Num'] >= 2100)]
both_1800_one_2100.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32740 entries, Game11 to Game1000055
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BlackElo       32740 non-null  object
 1   WhiteElo       32740 non-null  object
 2   moves          32740 non-null  object
 3   move_num       32740 non-null  object
 4   winner         32740 non-null  object
 5   Black_Elo_Num  32740 non-null  int64 
 6   White_Elo_Num  32740 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.0+ MB


In [19]:
# Pickle data
with open('2014_09_over_1800_one_2100_chess_data.pickle', 'wb') as to_write:
    pickle.dump(both_1800_one_2100, to_write)

In [20]:
# Save data set as csv
both_1800_one_2100.to_csv('2014_09_over_1800_one_2100_chess_data.csv')